In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [2]:
US = pd.read_csv('Data/Capstone/US-ok.csv')
CA = pd.read_csv('Data/Capstone/CA-ok.csv')
MX = pd.read_csv('Data/Capstone/MX-ok.csv')
DE = pd.read_csv('Data/Capstone/DE-ok.csv')
FR = pd.read_csv('Data/Capstone/FR-ok.csv')
GB = pd.read_csv('Data/Capstone/GB-ok.csv')
IN = pd.read_csv('Data/Capstone/IN-ok.csv')
KR = pd.read_csv('Data/Capstone/KR-ok.csv')
JP = pd.read_csv('Data/Capstone/JP-ok.csv')
RU = pd.read_csv('Data/Capstone/RU-ok.csv')


In [3]:
WC = pd.concat([CA, US, MX, DE, FR, GB, IN, JP, RU, KR], axis = 0)
WC.corr()

,category_id,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,assignable
category_id,1.000000,-0.133845,-0.152176,-0.034830,-0.072457,0.029289,0.015057,-0.026897,-0.161251
views,-0.133845,1.000000,0.780728,0.422737,0.516257,-0.006420,-0.010169,0.007639,0.006157
likes,-0.152176,0.780728,1.000000,0.448149,0.791881,-0.027853,-0.031377,0.006921,0.010212
dislikes,-0.034830,0.422737,0.448149,1.000000,0.705616,-0.007445,-0.012646,0.005538,0.003208
comment_count,-0.072457,0.516257,0.791881,0.705616,1.000000,-0.025147,-0.020070,0.003249,0.007328
comments_disabled,0.029289,-0.006420,-0.027853,-0.007445,-0.025147,1.000000,0.289881,-0.003825,-0.000735
ratings_disabled,0.015057,-0.010169,-0.031377,-0.012646,-0.020070,0.289881,1.000000,-0.003516,0.006528
video_error_or_removed,-0.026897,0.007639,0.006921,0.005538,0.003249,-0.003825,-0.003516,1.000000,0.001330
assignable,-0.161251,0.006157,0.010212,0.003208,0.007328,-0.000735,0.006528,0.001330,1.000000


In [4]:
WC.isna().sum()
World = WC.copy()

In [5]:
import time
World['trending_date'] = World.trending_date.apply(lambda date : '20' + str(date))
World['trending_date'] = World.trending_date.apply(lambda date : str(date).replace('.', '/'))
# Converting Trending date and publish time to datetime format
World.trending_date = pd.to_datetime(World['trending_date'], format = '%Y/%d/%m')
World.publish_time = pd.to_datetime(World['publish_time'])

World['NoTags'] = [len(tag.split('|')) for tag in World['tags']]
World['NoWord'] = [len(title.split(' ')) for title in World['title']]
World['NoChar'] = [len(title.strip(' ')) for title in World['title']]
World['TMonths'] = World['trending_date'].dt.month.astype(object)
World['PMonths'] = World['publish_time'].dt.month.astype(object)

World.drop(['video_id', 'category_id', 'thumbnail_link', 'description', 'title', 'channel_title', 'tags'], axis = 1, inplace = True )
World.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360662 entries, 0 to 31981
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   trending_date           360662 non-null  datetime64[ns]     
 1   publish_time            360662 non-null  datetime64[ns, UTC]
 2   views                   360662 non-null  int64              
 3   likes                   360662 non-null  int64              
 4   dislikes                360662 non-null  int64              
 5   comment_count           360662 non-null  int64              
 6   comments_disabled       360662 non-null  bool               
 7   ratings_disabled        360662 non-null  bool               
 8   video_error_or_removed  360662 non-null  bool               
 9   category                360662 non-null  object             
 10  assignable              360662 non-null  bool               
 11  NoTags                  360

In [6]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['category', 'TMonths', 'PMonths'])
NAbe = encoder.fit_transform(World)

In [7]:
NAbe = pd.get_dummies(NAbe, drop_first = True)
NAbe.shape

(360662, 28)

In [8]:
Xn = NAbe.drop(['views','trending_date', 'publish_time'], axis= 1)
Yn = NAbe['views']

In [9]:
from sklearn.feature_selection import SelectKBest, f_regression
feat = SelectKBest(score_func=f_regression, k='all')
Selectd_X = feat.fit(Xn, Yn)
#print(Selectd_X.scores_)
score = pd.DataFrame(Selectd_X.scores_)
feat = pd.DataFrame(Xn.columns)
FeatScores = pd.concat([feat, score], axis =1)
FeatScores.columns=['Features','Score']
print(FeatScores.nlargest(NAbe.shape[1],'Score'))

Features          Score
0                    likes  563011.278231
2            comment_count  131051.718805
1                 dislikes   78476.432484
9               category_3    3057.149703
15                  NoChar    2291.217870
11              category_5    1999.204541
10              category_4    1982.106420
17               TMonths_1     744.992897
8               category_2     622.411517
21               PMonths_1     443.175054
24               PMonths_4     212.969437
14                  NoWord     168.160843
16               TMonths_0     160.335096
23               PMonths_3     102.909552
4         ratings_disabled      37.297293
5   video_error_or_removed      21.045139
13                  NoTags      19.854323
3        comments_disabled      14.864239
12              assignable      13.670478
19               TMonths_3       6.991980
18               TMonths_2       3.697194
7               category_1       2.004833
22               PMonths_2       0.085381


In [10]:
NAbe1 = NAbe[FeatScores['Features'][FeatScores['Score'] > 2]]
NAbe1['views'] = NAbe['views'] 
NAbe1 = NAbe1[NAbe1['views'] >= 5000000]
#FeatScores['Features'][FeatScores['Score'] > 2]

In [11]:
Xn = NAbe1.drop('views', axis =1)
Yn = np.log(NAbe1['views'])

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(Xn)
y = sc.fit_transform(Yn.values.reshape(-1, 1))

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(Xn, Yn, test_size = 0.3, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

model1 = lm.fit(train_X, train_y)
y_pred = lm.predict(test_X)

from sklearn.metrics import mean_squared_error as mse
k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.52, 0.27)

In [13]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=200, max_depth= 100)
model2 = rfr.fit(train_X, train_y)
y_pred = rfr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.11, 0.01)

In [14]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', degree = 3)
model3 = svr.fit(train_X, train_y)
y_pred = svr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.48, 0.23)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
params = {'n_neighbors':list(range(1,21))}
knn = KNeighborsRegressor()
model = GridSearchCV(knn, params, cv=5)
model.fit(train_X, train_y)
print(model.best_params_)

y_pred2 = model.predict(test_X)
k = mse(test_y, y_pred2)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

{'n_neighbors': 2}


(0.48, 0.1)

In [16]:
Q1 = NAbe1.quantile(0.25)
Q3 = NAbe1.quantile(0.75)
IQR = Q3 - Q1
NAbe2 = NAbe1[~((NAbe1 < (Q1 - 1.5 * IQR)) |(NAbe1 > (Q3 + 1.5 * IQR))).any(axis=1)]

In [17]:
Xn = NAbe2.drop('views', axis =1)
Yn = np.log(NAbe2['views'])

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(Xn)
y = sc.fit_transform(Yn.values.reshape(-1, 1))

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(Xn, Yn, test_size = 0.3, random_state=42)

In [18]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

model5 = lm.fit(train_X, train_y)
y_pred = lm.predict(test_X)

from sklearn.metrics import mean_squared_error as mse
k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.39, 0.15)

In [19]:
rfr = RandomForestRegressor(n_estimators=200, max_depth= 100)
model6 = rfr.fit(train_X, train_y)
y_pred = rfr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.11, 0.01)

In [20]:
svr = SVR(kernel='rbf', degree = 3)
model7 = svr.fit(train_X, train_y)
y_pred = svr.predict(test_X)

k = mse(y_pred, test_y)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

(0.4, 0.16)

In [21]:
params = {'n_neighbors':list(range(1,21))}
knn = KNeighborsRegressor()
model8 = GridSearchCV(knn, params, cv=5)
model8.fit(train_X, train_y)
print(model.best_params_)

y_pred2 = model.predict(test_X)
k = mse(test_y, y_pred2)
rsme = mse(y_pred, test_y)**(1/2)
round(rsme, 2), round(k, 2)

{'n_neighbors': 2}


(0.4, 0.05)